In [1]:
import numpy as np
import numpy_financial as npf
import pandas as pd

def bond_price(face_value, coupon_rate, periods, interest_rate):
    coupon = face_value * coupon_rate
    discount_factor = 1 / (1 + interest_rate)
    
    price = (coupon * (1 - discount_factor ** periods) / interest_rate) + (face_value * discount_factor ** periods)
    return price

def bond_yield(face_value, coupon_rate, periods, price):
    coupon = face_value * coupon_rate
    
    try:
        ytm = npf.irr([-price] + [coupon] * (periods - 1) + [face_value + coupon])
    except Exception as e:
        print(f"Error calculating IRR: {e}")
        ytm = np.nan
    return ytm

def bond_duration(face_value, coupon_rate, periods, interest_rate):
    coupon = face_value * coupon_rate
    discount_factor = 1 / (1 + interest_rate)
    
    weighted_cash_flows = [(t * coupon * discount_factor ** t) for t in range(1, periods + 1)]
    weighted_cash_flows.append(periods * face_value * discount_factor ** periods)
    
    price = bond_price(face_value, coupon_rate, periods, interest_rate)
    duration = sum(weighted_cash_flows) / price
    
    return duration

def bond_convexity(face_value, coupon_rate, periods, interest_rate):
    coupon = face_value * coupon_rate
    discount_factor = 1 / (1 + interest_rate)
    
    weighted_cash_flows = [(t * (t + 1) * coupon * discount_factor ** (t + 2)) for t in range(1, periods + 1)]
    weighted_cash_flows.append(periods * (periods + 1) * face_value * discount_factor ** (periods + 2))
    
    price = bond_price(face_value, coupon_rate, periods, interest_rate)
    convexity = sum(weighted_cash_flows) / (price * (1 + interest_rate) ** 2)
    
    return convexity

def modified_duration(duration, interest_rate):
    return duration / (1 + interest_rate)

def effective_duration(price_up, price_down, delta_rate, price):
    return (price_down - price_up) / (2 * price * delta_rate)

def effective_convexity(price_up, price_down, price, delta_rate):
    return (price_down + price_up - 2 * price) / (price * delta_rate ** 2)

def dollar_duration(price, modified_duration):
    return price * modified_duration

def credit_spread(risk_free_rate, yield_to_maturity):
    return yield_to_maturity - risk_free_rate

def adjusted_spread(market_yield, risk_free_rate):
    return market_yield - risk_free_rate

# Example usage
face_value = 2000  # Face value of the bond
coupon_rate = 0.04  # 4% annual coupon rate
periods = 15  # 15 years to maturity
interest_rates = [0.01, 0.025, 0.035, 0.045, 0.055, 0.065]  # Interest rate scenarios
delta_rate = 0.002  # Small change in interest rate for effective duration/convexity

# Additional inputs for market conditions
current_market_price = 1980  # Current market price of the bond
market_yield = 0.042  # Market yield (4.2%)
risk_free_rate = 0.03  # Risk-free rate for credit and adjusted spread calculation

# Prepare the results in a list of dictionaries
results = []

for interest_rate in interest_rates:
    price = bond_price(face_value, coupon_rate, periods, interest_rate)
    ytm = bond_yield(face_value, coupon_rate, periods, price)
    duration = bond_duration(face_value, coupon_rate, periods, interest_rate)
    convexity = bond_convexity(face_value, coupon_rate, periods, interest_rate)
    mod_duration = modified_duration(duration, interest_rate)
    price_up = bond_price(face_value, coupon_rate, periods, interest_rate + delta_rate)
    price_down = bond_price(face_value, coupon_rate, periods, interest_rate - delta_rate)
    eff_duration = effective_duration(price_up, price_down, delta_rate, price)
    eff_convexity = effective_convexity(price_up, price_down, price, delta_rate)
    dollar_dur = dollar_duration(price, mod_duration)
    spread = credit_spread(risk_free_rate, ytm)
    adj_spread = adjusted_spread(market_yield, risk_free_rate)
    
    results.append({
        "Interest Rate": f"{interest_rate:.2%}",
        "Bond Price": f"${price:.2f}",
        "Yield to Maturity": f"{ytm:.2%}",
        "Macaulay Duration": f"{duration:.2f}",
        "Modified Duration": f"{mod_duration:.2f}",
        "Effective Duration": f"{eff_duration:.2f}",
        "Convexity": f"{convexity:.2f}",
        "Effective Convexity": f"{eff_convexity:.2f}",
        "Dollar Duration": f"${dollar_dur:.2f}",
        "Credit Spread": f"{spread:.2%}",
        "Current Market Price": f"${current_market_price:.2f}",
        "Market Yield": f"{market_yield:.2%}",
        "Adjusted Spread": f"{adj_spread:.2%}"
    })

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Save the DataFrame to an Excel file with proper file name and extension
excel_filename = r'C:\Dhananjay\Academics\Projects\bond_analysis_updated.xlsx'
df.to_excel(excel_filename, index=False)



In [2]:
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment
from openpyxl.utils import get_column_letter

# Load the workbook and select the active worksheet
wb = load_workbook(excel_filename)
ws = wb.active

# Auto-adjust column widths
for col in ws.columns:
    max_length = 0
    column = col[0].column_letter  # Get the column letter
    for cell in col:
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))
    ws.column_dimensions[column].width = max_length + 2  # Adjust for padding

# Make the first row bold and center-aligned
for cell in ws[1]:
    cell.font = Font(bold=True)
    cell.alignment = Alignment(horizontal="center", vertical="center")

# Define the red font style for negative values
red_font = Font(color="FF0000")

# Apply the red font style to all relevant cells in the DataFrame
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, max_col=ws.max_column):
    for cell in row:
        cell.alignment = Alignment(horizontal="center", vertical="center")
        if isinstance(cell.value, str) and '-' in cell.value:
            # Remove the '-' symbol
            cell_value = cell.value.replace('-', '')
            
            # Set the cell value and style
            cell.value = f'({cell_value})'
            cell.font = red_font

# Save the formatted Excel file
wb.save(excel_filename)

print(f"Formatted results saved to {excel_filename}")


Formatted results saved to C:\Dhananjay\Academics\Projects\bond_analysis_updated.xlsx


In [3]:
import pandas as pd
import matplotlib.pyplot as plt

def calculate_risk_metrics(face_value, coupon_rate, periods, interest_rates, current_price, risk_free_rate):
    results = []
    for rate in interest_rates:
        price = bond_price(face_value, coupon_rate, periods, rate)
        ytm = bond_yield(face_value, coupon_rate, periods, price)
        duration = bond_duration(face_value, coupon_rate, periods, rate)
        convexity = bond_convexity(face_value, coupon_rate, periods, rate)
        spread = credit_spread(risk_free_rate, ytm)
        
        risk_metrics = {
            "Interest Rate": rate,
            "Bond Price": price,
            "Yield to Maturity": ytm,
            "Duration": duration,
            "Convexity": convexity,
            "Credit Spread": spread
        }
        results.append(risk_metrics)
    
    return results

def normalize_metric(value, min_value, max_value):
    if max_value == min_value:
        return 0
    return (value - min_value) / (max_value - min_value)

def compute_weighted_risk_indicator(metrics, weights):
    normalized_metrics = {
        key: normalize_metric(value, min(metrics[key] for metrics in metrics_list), 
                                    max(metrics[key] for metrics in metrics_list))
        for key, value in metrics.items()
    }
    weighted_sum = sum(normalized_metrics.get(key, 0) * weights.get(key, 1) for key in normalized_metrics)
    return weighted_sum

def risk_indicator(metrics_list, weights):
    indicators = [compute_weighted_risk_indicator(metrics, weights) for metrics in metrics_list]
    return indicators

def generate_report(results, weights):
    df = pd.DataFrame(results)
    df['Risk Indicator'] = risk_indicator(results, weights)
    df.to_excel('risk_indicator_report.xlsx', index=False)
    
    # Plotting
    df.plot(x='Interest Rate', y='Risk Indicator', kind='line')
    plt.title('Risk Indicator vs Interest Rate')
    plt.xlabel('Interest Rate')
    plt.ylabel('Risk Indicator')
    plt.grid(True)
    plt.show()

# Example usage
face_value = 2000
coupon_rate = 0.04
periods = 15
interest_rates = [0.01, 0.025, 0.035, 0.045, 0.055, 0.065]
current_price = 1980
risk_free_rate = 0.03

results = calculate_risk_metrics(face_value, coupon_rate, periods, interest_rates, current_price, risk_free_rate)

weights = {
    "Bond Price": 0.2,
    "Yield to Maturity": 0.3,
    "Duration": 0.2,
    "Convexity": 0.2,
    "Credit Spread": 0.1
}

def generate_report(results, weights):
    df = pd.DataFrame(results)
    df.to_excel('risk_indicator_report.xlsx', index=False)
    df.plot(x='Interest Rate', y='Risk Indicator', kind='line')
    plt.title('Risk Indicator vs Interest Rate')
    plt.xlabel('Interest Rate')
    plt.ylabel('Risk Indicator')
    plt.show()


In [4]:
from scipy.optimize import minimize

# Bond specifications (example)
bond_list = [
    {"face_value": 1000, "coupon_rate": 0.05, "maturity_years": 10, "yield_rate": 0.04},
    {"face_value": 1000, "coupon_rate": 0.04, "maturity_years": 8, "yield_rate": 0.035},
    {"face_value": 1000, "coupon_rate": 0.06, "maturity_years": 12, "yield_rate": 0.045},
    # Add more bonds as needed
]

# Target portfolio parameters
desired_duration = 9  # Target portfolio duration
desired_convexity = 40.0  # Target portfolio convexity
desired_spread = 0.01  # Target portfolio credit spread

def calculate_bond_price(face_value, coupon_rate, periods, discount_rate):
    coupon = face_value * coupon_rate
    discount_factor = 1 / (1 + discount_rate)
    price = (coupon * (1 - discount_factor ** periods) / discount_rate) + (face_value * discount_factor ** periods)
    return price

def calculate_bond_duration(face_value, coupon_rate, periods, discount_rate):
    coupon = face_value * coupon_rate
    discount_factor = 1 / (1 + discount_rate)
    weighted_cash_flows = [(t * coupon * discount_factor ** t) for t in range(1, periods + 1)]
    weighted_cash_flows.append(periods * face_value * discount_factor ** periods)
    price = calculate_bond_price(face_value, coupon_rate, periods, discount_rate)
    duration = sum(weighted_cash_flows) / price
    return duration

def calculate_bond_convexity(face_value, coupon_rate, periods, discount_rate):
    coupon = face_value * coupon_rate
    discount_factor = 1 / (1 + discount_rate)
    weighted_cash_flows = [(t * (t + 1) * coupon * discount_factor ** (t + 2)) for t in range(1, periods + 1)]
    weighted_cash_flows.append(periods * (periods + 1) * face_value * discount_factor ** (periods + 2))
    price = calculate_bond_price(face_value, coupon_rate, periods, discount_rate)
    convexity = sum(weighted_cash_flows) / (price * (1 + discount_rate) ** 2)
    return convexity

def calculate_credit_spread(risk_free_rate, bond_yield):
    return bond_yield - risk_free_rate

def evaluate_portfolio(weights, bond_list, desired_duration, desired_convexity, desired_spread, risk_free_rate=0.03):
    portfolio_duration = sum(weights[i] * calculate_bond_duration(bond_list[i]['face_value'], bond_list[i]['coupon_rate'], bond_list[i]['maturity_years'], bond_list[i]['yield_rate']) for i in range(len(bond_list)))
    portfolio_convexity = sum(weights[i] * calculate_bond_convexity(bond_list[i]['face_value'], bond_list[i]['coupon_rate'], bond_list[i]['maturity_years'], bond_list[i]['yield_rate']) for i in range(len(bond_list)))
    portfolio_spread = sum(weights[i] * calculate_credit_spread(risk_free_rate, bond_list[i]['yield_rate']) for i in range(len(bond_list)))
    
    # Calculate the squared differences from target metrics
    duration_diff = ((portfolio_duration - desired_duration) / desired_duration) ** 2
    convexity_diff = ((portfolio_convexity - desired_convexity) / desired_convexity) ** 2
    spread_diff = ((portfolio_spread - desired_spread) / desired_spread) ** 2
    
    # Objective function to minimize
    return duration_diff + convexity_diff + spread_diff

# Initial guess (equal allocation)
initial_allocation = np.array([1 / len(bond_list)] * len(bond_list))

# Set minimum and maximum allocation constraints for each bond
min_allocation = 0.05  # Minimum allocation for any bond
max_allocation = 0.50  # Maximum allocation for any bond

# Define bounds for each bond
allocation_bounds = [(min_allocation, max_allocation) for _ in range(len(bond_list))]

# Constraints: allocations must sum to 1
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# Optimize the portfolio with constraints
optimization_result = minimize(evaluate_portfolio, initial_allocation, args=(bond_list, desired_duration, desired_convexity, desired_spread), bounds=allocation_bounds, constraints=constraints)

# Optimal allocations rounded to two decimal places
optimal_allocations = np.round(optimization_result.x, 2)

# Output the results
print("Optimal Allocations:", optimal_allocations)

# Calculate the final portfolio metrics using the rounded allocations
final_portfolio_duration = sum(optimal_allocations[i] * calculate_bond_duration(bond_list[i]['face_value'], bond_list[i]['coupon_rate'], bond_list[i]['maturity_years'], bond_list[i]['yield_rate']) for i in range(len(bond_list)))
final_portfolio_convexity = sum(optimal_allocations[i] * calculate_bond_convexity(bond_list[i]['face_value'], bond_list[i]['coupon_rate'], bond_list[i]['maturity_years'], bond_list[i]['yield_rate']) for i in range(len(bond_list)))
final_portfolio_spread = sum(optimal_allocations[i] * calculate_credit_spread(0.03, bond_list[i]['yield_rate']) for i in range(len(bond_list)))

print(f"Portfolio Duration: {final_portfolio_duration:.2f}")
print(f"Portfolio Convexity: {final_portfolio_convexity:.2f}")
print(f"Portfolio Spread: {final_portfolio_spread:.4f}")


Optimal Allocations: [0.45 0.5  0.05]
Portfolio Duration: 7.65
Portfolio Convexity: 63.01
Portfolio Spread: 0.0078
